<a href="https://colab.research.google.com/github/devShaurya/Summer-Intern/blob/master/FloorNetFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone the floorNet repo

In [ ]:
!git clone https://github.com/art-programmer/FloorNet.git

In [ ]:
%cd FloorNet

## Download Dataset

Use [this](https://drive.google.com/open?id=16lyX_xTiALUzKyst86WJHlhpTDr8XPF_) or [this](https://mega.nz/#F!5yQy0b5T!ykkR4dqwGO9J5EwnKT_GBw) link to download the dataset files. As you will see, the images of floorplans are stored in .tfrecords format. 

The readme of FloorNet suggests to read RecordWriterTango.py and some tutorials for understanding the .tfrecords format. But we don't need to read those. Read [this](https://www.tensorflow.org/tutorials/load_data/tfrecord) tutorial to learn more understand about tfrecords format.

If you read that tutorial, or you already know about tfrecords format than you will understand that tfrecords are used to save data in key-value pairs. Hence we need to know the keys for extracting the code out of these tfrecords. 

RecordReader.py file is the one which helps in reading the data in FloorNet pipeline so we can use it for extracting data. It also mentions the key-value pairs too

After downloading the data, save it under "data" folder of FloorNet repo and follow this notebook for extraction of the images

## Extraction of Dataset

In [ ]:
### Run this cell first before other cells else some of the next cells give weird errors
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:
from RecordReader import *
from augmentation_tf import *
from floorplan_utils import *
from utils import *

In [ ]:

filenames=['data/Tango_train.tfrecords']

augmentation=""

batchSize=6

readImageFeatures=True

converCorners=True

In [ ]:
def parse_fn(example, augmentation='', readImageFeatures = True, convertCorners=True, kernelSize=11):
    if readImageFeatures:
        features = tf.parse_single_example(
            example,
            # Defaults are not specified since both keys are required.
            features={
                'image_path': tf.FixedLenFeature([], tf.string),
                'points': tf.FixedLenFeature([NUM_POINTS * (NUM_INPUT_CHANNELS - 1)], tf.float32),
                'point_indices': tf.FixedLenFeature([NUM_POINTS], tf.int64),
                'corner': tf.FixedLenFeature([MAX_NUM_CORNERS * 3], tf.int64),
                'num_corners': tf.FixedLenFeature([], tf.int64),
                'icon': tf.FixedLenFeature([], tf.string),
                'room': tf.FixedLenFeature([], tf.string),
                'image': tf.FixedLenFeature(sum([size * size * numChannels for size, numChannels in list(zip(SIZES, NUM_CHANNELS))[1:]]), tf.float32),
                'flags': tf.FixedLenFeature([2], tf.int64),
            })
    else:
        features = tf.parse_single_example(
            example,
            # Defaults are not specified since both keys are required.
            features={
                'image_path': tf.FixedLenFeature([], tf.string),
                'points': tf.FixedLenFeature([NUM_POINTS * (NUM_INPUT_CHANNELS - 1)], tf.float32),
                'point_indices': tf.FixedLenFeature([NUM_POINTS], tf.int64),
                'corner': tf.FixedLenFeature([MAX_NUM_CORNERS * 3], tf.int64),
                'num_corners': tf.FixedLenFeature([], tf.int64),
                'icon': tf.FixedLenFeature([], tf.string),
                'room': tf.FixedLenFeature([], tf.string),
                'flags': tf.FixedLenFeature([2], tf.int64),
            })
        pass

    points = tf.reshape(features['points'], (NUM_POINTS, NUM_INPUT_CHANNELS - 1))
    point_indices = tf.cast(features['point_indices'], tf.int32)
    #point_indices = features['point_indices']

    corners = tf.cast(tf.reshape(features['corner'], [MAX_NUM_CORNERS, 3]), tf.int32)
    numCorners = features['num_corners']
    corners = corners[:numCorners]
    iconSegmentation = tf.reshape(tf.decode_raw(features['icon'], tf.uint8), (HEIGHT, WIDTH, 1))
    roomSegmentation = tf.reshape(tf.decode_raw(features['room'], tf.uint8), (HEIGHT, WIDTH, 1))
    heatmaps = tf.stack([iconSegmentation, roomSegmentation], axis=0)

    if readImageFeatures:
        imageFeature = features['image']
        imageFeatures = {}
        offset = 0
#         print(SIZES,NUM_CHANNELS)
        new_zip=list(zip(SIZES, NUM_CHANNELS))
#         print(type(new_zip))
        for index, (size, numChannels) in enumerate(new_zip[1:]):
            imageFeatures[index] = tf.reshape(imageFeature[offset:offset + size * size * numChannels], (size, size, numChannels))
            offset += size * size * numChannels
            continue
    else:
        imageFeatures = {}
        pass

    flags = features['flags']
    if 'w' in augmentation:
        point_indices, corners, heatmaps = tf.cond(tf.logical_or(tf.equal(flags[0], 0), tf.equal(flags[0], 4)), lambda: augmentWarping(point_indices, corners, heatmaps, gridStride=32, randomScale=2), lambda: (point_indices, corners, heatmaps))
        #point_indices, corners, heatmaps = augmentWarping(point_indices, corners, heatmaps, gridStride=32, randomScale=4)
        pass
    if 's' in augmentation:
        points, point_indices, corners, heatmaps, imageFeatures = augmentScaling(points, point_indices, corners, heatmaps, imageFeatures)
        pass
    if 'f' in augmentation:
        points, point_indices, corners, heatmaps, imageFeatures = augmentFlipping(points, point_indices, corners, heatmaps, imageFeatures)
        pass

    iconSegmentation = tf.cast(tf.squeeze(heatmaps[0]), tf.int32)
    roomSegmentation = tf.cast(tf.squeeze(heatmaps[1]), tf.int32)

    roomSegmentation = tf.minimum(roomSegmentation, NUM_ROOMS - 1)

    # point_indices_stack = getCoarseIndicesMaps(point_indices, WIDTH, HEIGHT, 0)

    corners = tf.reshape(tf.concat([corners, tf.zeros((MAX_NUM_CORNERS - tf.shape(corners)[0], 3), dtype=tf.int32)], axis=0), (MAX_NUM_CORNERS, 3))
    if convertCorners:
        cornerSegmentation = tf.stack([tf.sparse_to_dense(tf.stack([corners[:, 1], corners[:, 0]], axis=1), (HEIGHT, WIDTH), corners[:, 2], validate_indices=False)], axis=0)
        cornerHeatmaps = tf.one_hot(cornerSegmentation, depth=NUM_CORNERS + 1, axis=-1)[:, :, :, 1:]
        #kernel_size = kernelSize
        #neighbor_kernel_array = disk(kernel_size)
        #neighbor_kernel = tf.constant(neighbor_kernel_array.reshape(-1), shape=neighbor_kernel_array.shape, dtype=tf.float32)
        #neighbor_kernel = tf.reshape(neighbor_kernel, [kernel_size, kernel_size, 1, 1])
        #cornerHeatmaps = tf.nn.depthwise_conv2d(cornerHeatmaps, tf.tile(neighbor_kernel, [1, 1, NUM_CORNERS, 1]), strides=[1, 1, 1, 1], padding='SAME')
        corners = tf.cast(cornerHeatmaps > 0.5, tf.float32)

    # cornerSegmentation = tf.sparse_to_dense(tf.stack([corners[:, 1], corners[:, 0]], axis=1), (HEIGHT, WIDTH), corners[:, 2], validate_indices=False)
    # cornerHeatmaps = tf.one_hot(cornerSegmentation, depth=NUM_CORNERS, axis=-1)
    # kernel = tf.tile(tf.expand_dims(tf.constant(disk(11)), -1), [1, 1, NUM_CORNERS])
    # cornerHeatmaps = tf.nn.dilation2d(tf.expand_dims(cornerHeatmaps, 0), kernel, [1, 1, 1, 1], [1, 1, 1, 1], 'SAME')[0]

    imagePath = features['image_path']

    points = tf.concat([points, tf.ones((NUM_POINTS, 1))], axis=1)

    if readImageFeatures:
        input_dict = {'points': points, 'point_indices': point_indices, 'image_features': imageFeatures, 'image_path': imagePath, 'flags': flags}
    else:
        input_dict = {'points': points, 'point_indices': point_indices, 'image_path': imagePath, 'flags': flags}
        pass
    gt_dict = {'corner': corners, 'icon': iconSegmentation, 'room': roomSegmentation, 'num_corners': numCorners}
    return input_dict, gt_dict

In [ ]:
raw_dataset = tf.data.TFRecordDataset(filenames[0])
parsed_dataset = raw_dataset.map(parse_fn)
def saveImages(typed):
    num=1 
    for i in parsed_dataset:
        cv2.imwrite("./data/train/GTv3/{}s_mask/{}_mask_{}.bmp".format(typed,typed,num)
                    ,i[1][typed].numpy())#(.format(typed,typed,num)) 
#         break
        num+=1

In [ ]:
saveImages("room") 
saveImages("icon")

## Extractiong boundary and wall masks
After extracting the dataset, run the cells below.

In [ ]:
def saveNonSegmentedImages(typed):
    if typed=='room':
        for i in range(1,136):
            image=np.array(Image.open("./data/train/GTv3/{}s_mask/{}_mask_{}.bmp".format(typed,typed,i)))
            newImage=image.copy()
            newImage[image==0]=255
            newImage[image!=15]=255
            newImage[image==15]=0
            cv2.imwrite("./data/train/NonSegmentv1/{}s/{}_{}.bmp".format(typed,typed,i),newImage)

In [ ]:
saveNonSegmentedImages("room")

In [ ]:
def saveBoundary():
    for ind in range(1,136):
        origImage=np.array(Image.open("./data/train/GTv3/rooms_mask/room_mask_{}.bmp".format(ind)))
        floorImage=np.ones((256,256))
        for i in range(2,254):
            for j in range(2,254):
                flag=0
                temp=origImage[i-2:i+3,j-2:j+3]
                tempCount=np.unique(temp,return_counts=1)
                if(i-3>=0 and origImage[i-3][j]==0) or (i+3<=255 and origImage[i+3][j]==0) or (j-3>=0 
                    and origImage[i][j-3]==0) or (j+3<=255 and origImage[i][j+3]==0):
                    flag=1
                if(flag==1 and temp[0][0]==15 and tempCount[1][0]==25):
                    floorImage[i-2:i+3,j-2:j+3]=0
        floorImage[floorImage==1]=255
        cv2.imwrite("./data/train/NonSegmentv1/onlyBoundary/boundary_{}.bmp".format(ind),floorImage)

In [ ]:
saveBoundary()